#Neural Signal Decoder (Simulated)

The Neural Signal Decoder (Simulated) project is a complete, end‑to‑end implementation of a neural time‑series decoding system inspired by Brain–Computer Interface (BCI) research. The system is designed to translate multichannel EEG‑like signals into discrete control actions using deep learning sequence models.

##System Architecture

The Neural Signal Decoder follows a modular pipeline architecture:


```NEURAL SIGNAL → PREPROCESSING → TEMPORAL ENCODER → CLASSIFIER → ACTION OUTPUT```


##ECE Signal

### Explanation of `simulate_eeg_sample` Function

This Python code defines a function `simulate_eeg_sample` which generates a simulated EEG-like signal based on a given `class_id`. It's designed to mimic neural signals with different frequency characteristics depending on the intended 'action' (represented by `class_id`).

#### Function Parameters:

*   **`class_id`**: An integer representing the 'action' or mental state (0 for 'Left', 1 for 'Right', 2 for 'Up', 3 for 'Down'). Each class is associated with a distinct frequency band.
*   **`num_channels`**: The number of simulated EEG channels.
*   **`num_timesteps`**: The number of data points (samples) in the time series for each channel.
*   **`sampling_rate`**: The number of samples per second (default is 256 Hz, common in EEG).

#### How the Signal is Generated (Mathematical Explanation):

1.  **Time Axis (`t`)**:

    ```python
    t = np.arange(num_timesteps) / sampling_rate
    ```

    *   `np.arange(num_timesteps)` creates an array of integers from `0` to `num_timesteps - 1`. These represent the sample indices.
    *   Dividing by `sampling_rate` converts these sample indices into actual time values in seconds. For example, if `sampling_rate` is 256, the first sample is at `t=0`, the second at `t=1/256`, the third at `t=2/256`, and so on.

2.  **Class-Dependent Frequency Bands (`freq_bands`)**:

    ```python
    freq_bands = {
        0: (7, 10),    # Left (e.g., Alpha band)
        1: (11, 14),   # Right (e.g., Low Beta band)
        2: (15, 20),   # Up (e.g., Mid Beta band)
        3: (20, 25)    # Down (e.g., High Beta band)
    }
    low_f, high_f = freq_bands[class_id]
    ```

    This dictionary maps each `class_id` to a specific frequency range (`low_f`, `high_f`). When generating the signal for a given `class_id`, the function picks a random frequency within this band.

3.  **Signal Generation per Channel**:

    The core of the simulation happens in a loop, generating a signal for each `num_channels`.

    *   **Frequency (`freq`) and Phase (`phase`) Selection**:

        ```python
        freq = np.random.uniform(low_f, high_f)
        phase = np.random.uniform(0, 2 * np.pi)
        ```

        For each channel, a random frequency (`freq`) is chosen uniformly from the `[low_f, high_f]` band. A random phase (`phase`) is also chosen uniformly between `0` and `2π` radians. This randomness helps make each channel slightly different, as would be expected in real EEG.

    *   **Sine Wave Generation (`wave`)**:

        ```python
        wave = np.sin(2 * np.pi * freq * t + phase)
        ```

        This is the mathematical formula for a pure sine wave:
        `A * sin(2πft + φ)`
        Where:
        *   `A` is the amplitude (implicitly 1 here).
        *   `f` is the `freq` chosen for the channel.
        *   `t` is the time axis array.
        *   `φ` is the `phase` chosen for the channel.

        This generates a clean oscillatory signal specific to the `class_id`'s frequency band.

    *   **Noise Generation (`noise`)**:

        ```python
        noise = np.random.normal(0, 0.3, size=num_timesteps)
        ```

        This adds random noise to the clean sine wave. `np.random.normal(0, 0.3, ...)` generates samples from a normal (Gaussian) distribution with:
        *   Mean (`μ`) = 0
        *   Standard Deviation (`σ`) = 0.3

        This simulates the inherent randomness and interference present in real-world biological signals.

    *   **Combined Signal**:

        ```python
        signal[ch] = wave + noise
        ```

        The final signal for each channel is the sum of the pure sine wave and the Gaussian noise. This creates a more realistic EEG-like waveform where the underlying brain activity (the sine wave) is partially obscured by noise.

#### Return Value (`signal`)

*   The function returns a `numpy.ndarray` with the shape `(num_channels, num_timesteps)`, where each row corresponds to a different channel and each column to a time step. This array contains the simulated EEG data.

In [ ]:
import numpy as np

def simulate_eeg_sample(
    class_id: int,
    num_channels: int,
    num_timesteps: int,
    sampling_rate: int = 256
):
    """
    Returns:
        signal: np.ndarray of shape (num_channels, num_timesteps)
    """

    # Time axis
    t = np.arange(num_timesteps) / sampling_rate

    # Class-dependent frequency bands
    freq_bands = {
        0: (7, 10),    # Left
        1: (11, 14),   # Right
        2: (15, 20),   # Up
        3: (20, 25)    # Down
    }

    low_f, high_f = freq_bands[class_id]

    signal = np.zeros((num_channels, num_timesteps))

    for ch in range(num_channels):
        freq = np.random.uniform(low_f, high_f)
        phase = np.random.uniform(0, 2 * np.pi)

        wave = np.sin(2 * np.pi * freq * t + phase)
        noise = np.random.normal(0, 0.3, size=num_timesteps)

        signal[ch] = wave + noise

    return signal


### Module 2: Signal Pre-processing - Connecting Code to Documentation

The `normalize_channels` and `window_signal` functions are integral to this stage.

1.  **`normalize_channels` Function**: This function directly addresses the 'PREPROCESSING' step in your overall system architecture (`NEURAL SIGNAL → PREPROCESSING → TEMPORAL ENCODER → CLASSIFIER → ACTION OUTPUT`). It standardizes the signal across each channel, ensuring that variations in amplitude between different channels do not disproportionately affect downstream models. This normalization step is vital for robust model performance, especially with sensitive neural data.

2.  **`window_signal` Function**: Following normalization, the continuous pre-processed signal is then segmented into fixed-size 'windows' or 'epochs' using the `window_signal` function. This process is essential for sequence models, which typically require input data in discrete, fixed-length segments. By creating these windows, the function prepares the data for the 'TEMPORAL ENCODER' stage, allowing the model to learn patterns within specific timeframes of the neural activity.

### `normalize_channels` Function

This function takes a 2D NumPy array representing multi-channel signals (e.g., EEG data) and performs channel-wise standardization (Z-score normalization). This is a common preprocessing step to ensure that each channel contributes equally to subsequent analysis or model training, regardless of its original amplitude scale.

#### Function Parameters:

*   **`signal`**: A `numpy.ndarray` of shape `(C, T)`, where `C` is the number of channels and `T` is the number of time steps (samples).

#### How Normalization Works:

1.  **Iterate through Channels**: The function loops through each channel of the input `signal`.
2.  **Calculate Mean and Standard Deviation**: For each channel `c`:
    *   `mean = signal[c].mean()`: The average value of the signal across all time steps for that specific channel is calculated.
    *   `std = signal[c].std() + 1e-8`: The standard deviation of the signal for that channel is calculated. A small constant `1e-8` is added to the standard deviation to prevent division by zero in case a channel has no variance (i.e., all values are the same).
3.  **Apply Standardization**: The standardization formula is applied to each data point in the channel:
    *   `normalized[c] = (signal[c] - mean) / std`
    This transforms the data such that each channel will have a mean of approximately 0 and a standard deviation of approximately 1.

#### Return Value (`normalized_signal`)

*   A `numpy.ndarray` of the same shape `(C, T)` as the input `signal`, containing the standardized (normalized) signal data for all channels.


In [ ]:
def normalize_channels(signal: np.ndarray):
    """
    signal: np.ndarray of shape (C, T)

    returns:
        normalized_signal: np.ndarray of shape (C, T)
    """
    C, T = signal.shape
    normalized = np.zeros_like(signal)

    for c in range(C):
        mean = signal[c].mean()
        std = signal[c].std() + 1e-8
        normalized[c] = (signal[c] - mean) / std

    return normalized


### `window_signal` Function

This function takes a continuous multi-channel signal and divides it into overlapping or non-overlapping segments (windows) of a fixed size. This technique, known as "windowing" or "sliding window," is crucial in time-series analysis for preparing data for sequence models, allowing the model to process smaller, fixed-length chunks of the signal.

#### Function Parameters:

*   **`signal`**: A `numpy.ndarray` of shape `(C, T)`, where `C` is the number of channels and `T` is the total number of time steps.
*   **`window_size`**: An integer specifying the length of each window (number of time steps).
*   **`stride`**: An integer specifying the number of time steps to advance for the start of the next window. If `stride` equals `window_size`, the windows are non-overlapping. If `stride` is less than `window_size`, the windows overlap.

#### How Windowing Works:

1.  **Initialize**: An empty list `windows` is created to store the generated signal segments.
2.  **Slide the Window**: The function iterates from the beginning of the signal (`start = 0`) up to a point where a full `window_size` can still be extracted.
    *   The loop continues as long as `start` is such that `start + window_size` does not exceed the total number of time steps `T`.
    *   `start` is incremented by `stride` in each iteration, effectively sliding the window across the signal.
3.  **Extract Window**: In each iteration, a segment of the `signal` is extracted:
    *   `signal[:, start:end]` selects all channels (`:`) and the time steps from `start` to `end` (exclusive of `end`), where `end = start + window_size`.
    *   This extracted segment, which has the shape `(C, window_size)`, is appended to the `windows` list.
4.  **Stack Windows**: After all windows have been extracted, `np.stack(windows)` converts the list of 2D arrays into a single 3D NumPy array.

#### Return Value (`windows`)

*   A `numpy.ndarray` of shape `(N, C, window_size)`, where:
    *   `N` is the total number of windows created.
    *   `C` is the number of channels.
    *   `window_size` is the length of each individual window.

In [ ]:
def window_signal(signal: np.ndarray, window_size: int, stride: int):
    """
    signal: np.ndarray of shape (C, T)

    returns:
        windows: np.ndarray of shape (N, C, window_size)
    """
    C, T = signal.shape
    windows = []

    for start in range(0, T - window_size + 1, stride):
        end = start + window_size
        windows.append(signal[:, start:end])

    return np.stack(windows)
